In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pydeck as pdk

# 포스트 데이터 불러오기
df_post = pd.read_csv('../inputs/posts/euljiro.csv', index_col=0)

# 텍스트 없으면 drop
df_post = df_post.dropna()

In [2]:
df_post.head()

,date,location,text
0,2023-11-10T05:20:04.000Z,을지로 3가,.을지로의 상징이 되어버린 올디스😁⠀네온사인이 참 이쁘다
1,2023-11-07T12:29:08.000Z,을지로 3가,일본식 중화요리를 선보이는 을지로 신상 맛집 -각양각색의 맛집들이 모여 있는을지로 ...
2,2023-12-25T09:13:20.000Z,을지로 3가,🎄......#스냅사진 #스냅모델
3,2023-12-23T15:51:57.000Z,을지로 3가,블랜딩밀크가 들어가 고소하고 부드러운 라떼 맛집☕️을지로 인쇄골목 사이에 위치한 #...
4,2023-11-09T02:56:31.000Z,을지로 3가,손에서 컵을 못 놓겠네..☕️...이번에 착용한 데님은 학교 동아리에서 스프레이로 ...


In [3]:
# 12월로 데이터 필터링

df_post = df_post.loc[df_post.loc[:,'date'].str.contains('2023-12')]

In [4]:
# 상점 데이터 불러오기
gdf_store_euljiro = gpd.read_file('../inputs/stores/store_euljiro.geojson')

# 음식점만 필터링
gdf_store_euljiro = gdf_store_euljiro.loc[gdf_store_euljiro.loc[:,'상권업종대분류명'] == '음식']

# 상호명 없는 상점은 삭제
gdf_store_euljiro = gdf_store_euljiro.dropna(subset='상호명')

# 상호명과 geometry만 남기고 다 삭제 (현재 필요없는 컬럼들)
gdf_store_euljiro = gdf_store_euljiro.loc[:,['상호명', 'geometry']]

# 상호명: store_name으로 변경
gdf_store_euljiro = gdf_store_euljiro.rename(columns={'상호명':'store_name'})

In [5]:
gdf_store_euljiro.head()

,store_name,geometry
6,르템플,POINT (126.99240 37.56589)
7,낮한끼저녁한잔,POINT (126.98923 37.56513)
10,일품,POINT (126.98958 37.56533)
14,휘,POINT (126.99265 37.56306)
17,죽향,POINT (126.98938 37.56465)


In [6]:
def counting_post(store_name):
    '''
    상점 이름을 input으로 받아 posting 갯수를 세주는 함수

    input:
        store_name (string): 상호명
    output:
        posting_count (int): 포스팅 갯수

    '''

    # 상점 이름 을지인 것은 패스
    if store_name == '을지':
        return 0

    # 상호명이 한글자이면 그냥 0개로 취급
    if len(store_name) == 1:
        return 0

    # 포스트 중에서 상점 이름이 포함되어 있으면 좌표 추가
    posting_count = df_post.loc[df_post.loc[:,'text'].str.contains(store_name, regex=False)].shape[0]

    return posting_count


In [7]:
gdf_store_euljiro.loc[:,'posting_count'] = gdf_store_euljiro.loc[:,'store_name'].apply(counting_post)

In [8]:
# 포스팅 갯수 기준으로 정렬
gdf_store_euljiro = gdf_store_euljiro.sort_values(by='posting_count', ascending=False)

In [9]:
for i, row in gdf_store_euljiro.iterrows():
    print(row['store_name'], row['posting_count'])
    if row['posting_count'] == 0:
        break

또롱 20
을지로술집 19
갤러리 15
공간갑 4
베로나 4
효계 4
을지로라스베가스 3
부장님댁 3
타마고 3
코야 3
을지오뎅 3
을지1막 3
돈독 3
토리노라스베가스 3
우르르 2
어나더레벨 2
분카샤 2
82올리언즈 2
부지 2
주마등 2
광장 2
다래 2
일품 2
오덕장 2
을지빈 2
까이9 2
을지로밑 2
일단 2
만선호프 1
다크문 1
을지맥옥 1
애플 1
을지도 1
을지로보석 1
십분의일 1
을지로골목집 1
굿모닝 1
맨틱 1
줄리아 1
을지깐깐 1
부스 1
취향로3가 1
좋은날 1
금토일샴페인빠 1
혜민당 1
302호 1
라이너 1
만선 1
전주집 1
수선화 1
코너숍 1
르템플 1
드므 1
단칸 1
가야성 1
콸콸 1
남영동양문 1
영동 1
낭만 1
을지다락 1
을지로제 1
동강나루터 0


In [10]:
# 상점명 중 너무 보편적이여서 배제할 단어들을 우선 눈으로 리스팅 
list_stopwords = ['오늘', '충무', '을지로술집', '아트', '갤러리', '블루커피베이크', '현대', '선물', '연락', '디어', '충무로맛집', '광고', '포인트', '세운', '나무', '청계', '일단', '맛나', '가자', '무한', '힐링', '낭만', '인쇄소', '남산', '디엠']

In [11]:
# list_stopwords 안에 있는 상점들은 우선 데이터프레임에서 제외
gdf_store_euljiro = gdf_store_euljiro.loc[~gdf_store_euljiro.loc[:,'store_name'].isin(list_stopwords)]

# 포스팅 수가 0인 것들도 데이터프레임에서 제외
gdf_store_euljiro = gdf_store_euljiro.loc[gdf_store_euljiro.loc[:,'posting_count']>0]

In [12]:
# 약 372개 장소가 살아남음
gdf_store_euljiro

,store_name,geometry,posting_count
215,또롱,POINT (126.99055 37.56517),20
973,공간갑,POINT (126.99026 37.56549),4
238,베로나,POINT (126.99146 37.56547),4
77,효계,POINT (126.99113 37.56544),4
151,을지로라스베가스,POINT (126.99055 37.56517),3
582,부장님댁,POINT (126.99122 37.56539),3
408,타마고,POINT (126.98847 37.56670),3
1461,코야,POINT (126.99055 37.56517),3
106,을지오뎅,POINT (126.98992 37.56592),3
263,을지1막,POINT (126.99147 37.56532),3


In [13]:
gdf_store_euljiro.loc[:,'posting_count'].sum()

114

## pydeck으로 시각화 (탐색용)

In [14]:
# pydeck layer 설정

scatterplot_layer = pdk.Layer(
    'ScatterplotLayer',
    gdf_store_euljiro,
    pickable=True,
    opacity=0.3,
    stroked=False,
    filled=True,
    radius_scale=2,
    get_position='geometry.coordinates',
    get_radius='posting_count',
    get_fill_color=[255,140,0]
)

In [15]:
# Set the viewport location
view_state = pdk.ViewState(latitude=37.565, longitude=126.9915, zoom=15, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[scatterplot_layer], initial_view_state=view_state, tooltip={"text": "{store_name}\n{posting_count}"})
r.to_html("../output/maps/euljiro/scatterplot_layer.html")

In [16]:
heatmap_layer = pdk.Layer(
    "HeatmapLayer",
    data=gdf_store_euljiro,
    opacity=0.5,
    get_position='geometry.coordinates',
    aggregation=pdk.types.String("MEAN"),
    get_weight="posting_count",
)

In [17]:
r = pdk.Deck(
    layers=[heatmap_layer],
    initial_view_state=view_state
)

r.to_html("../output/maps/euljiro/heatmap_layer.html")